In [9]:
# get ratings
import a1_rtgs

# get the team ratings
d = a1_rtgs.bball_ratings("results.csv")

In [10]:
# make predictions for the rest of the games 
import a1_preds

preds = a1_preds.predictions("remaining_regular_season.csv",d)

In [11]:
# find final standings
import pandas as pd 
import numpy as np 

f_res = "results.csv"
df = pd.read_csv(f_res)
games = preds
df['ptsDiff'] = df['GH']-df['GA']
df = df.drop(['Date','Nothing','Overtimes','Notes','GA','GH'],axis=1)
final_df = pd.concat([df,games],sort=False).reset_index()
teams = list(set(df.home.unique()) & set(df.away.unique()))
points = { i : 0 for i in teams }
standings = []

for i in range(len(final_df)):
    if final_df['ptsDiff'][i] > 0:
        points[final_df['home'][i]] += 2
        points[final_df['away'][i]] += 1
    else:
        points[final_df['home'][i]] += 1
        points[final_df['away'][i]] += 2
        
points = {k: v for k, v in sorted(points.items(), key=lambda item: item[1],reverse=True)}

print("++++++++++++++++Final Standings++++++++++++++++")
ties = {}
for pair in points.items():
    if pair[1] not in ties.keys():
        ties[pair[1]] = []
    ties[pair[1]].append(pair[0])
for i in range(len(ties)):
    if len(list(ties.values())[i]) != 1:
        tie_dict = {i: [0,0] for i in list(ties.values())[i]}
        for g in range(len(final_df)):
            if final_df['home'][g] in tie_dict.keys() and final_df['away'][g] in tie_dict.keys():
                if final_df['ptsDiff'][g] > 0:
                    tie_dict[final_df['home'][g]][0] += 1
                else:
                    tie_dict[final_df['away'][g]][0] += 1
                tie_dict[final_df['home'][g]][1] += final_df['ptsDiff'][g]
                tie_dict[final_df['away'][g]][1] -= final_df['ptsDiff'][g]
        tie_stand = {k: v for k, v in sorted(tie_dict.items(), key=lambda item: item[1],reverse=True)}
        for t in range(len(tie_stand.keys())):
                print(list(tie_stand.keys())[t],list(ties.keys())[i])
                standings.append(list(tie_stand.keys())[t])
    else:
        print(list(ties.values())[i][0],list(ties.keys())[i])
        standings.append(list(ties.values())[i][0])
print("++++++++++++++++End Standings++++++++++++++++")
print()
print("Entering Playoffs: ",','.join(map(str,standings[0:8])))
print("Relegating: ", ','.join(map(str,standings[12:14])))

++++++++++++++++Final Standings++++++++++++++++
Panathinaikos OPAP 48
AEK Athens 44
Peristeri 42
Promitheas Patras 42
Lavrio B.C. 39
Ifaistos Limnou 37
Iraklis B.C. 36
Larisa 35
Rethymno Aegean 35
Kolossos Rodou 35
Ionikos Nikaias 34
P.A.O.K. 33
Aris 33
Panionios 32
++++++++++++++++End Standings++++++++++++++++

Entering Playoffs:  Panathinaikos OPAP,AEK Athens,Peristeri,Promitheas Patras,Lavrio B.C.,Ifaistos Limnou,Iraklis B.C.,Larisa
Relegating:  Aris,Panionios


In [12]:
# simulate a playoff series

def series_playoff(t1,t2,best_of, rtgs):
    if best_of == 5:
        W = 3
    else:
        if best_of == 3:
            W = 2
        else:
            print("Best of 3 or 5 series")
            return 
    wins = 0
    loses = 0
    for i in list(range(1,best_of+1)):
        if (i % 2) != 0:
            if np.random.normal(d['Home']+d[t1]-d[t2],12) > 0:
                wins+=1
            else:
                loses+=1
        else:
            if np.random.normal(d['Home']+d[t2]-d[t1],12) > 0:
                loses += 1
            else:
                wins += 1
        if wins == W:
            break
    return wins, loses

# 1st round

In [13]:
# first matchup
advance_to_second_round = []
t1 = standings[0]
t2 = standings[7]
wins, loses = series_playoff(t1,t2,3,d)
if wins == 2:
    advance_to_second_round.append(t1)
    print(t1,"wins series against ", t2,": ",wins,"-",loses)
else:
    print(t2,"wins series against ", t1,": ",loses,"-",wins)
    advance_to_second_round.append(t2)

#second matchup
t1 = standings[1]
t2 = standings[6]
wins, loses = series_playoff(t1,t2,3,d)
if wins == 2:
    advance_to_second_round.append(t1)
    print(t1,"wins series against ", t2,": ",wins,"-",loses)
else:
    print(t2,"wins series against ", t1,": ",loses,"-",wins)
    advance_to_second_round.append(t2)

#third matchup
t1 = standings[2]
t2 = standings[5]
wins, loses = series_playoff(t1,t2,3,d)
if wins == 2:
    advance_to_second_round.append(t1)
    print(t1,"wins series against ", t2,": ",wins,"-",loses)
else:
    print(t2,"wins series against ", t1,": ",loses,"-",wins)
    advance_to_second_round.append(t2)
    
#fourth matchup
t1 = standings[3]
t2 = standings[4]
wins, loses = series_playoff(t1,t2,3,d)
if wins == 2:
    advance_to_second_round.append(t1)
    print(t1,"wins series against ", t2,": ",wins,"-",loses)
else:
    print(t2,"wins series against ", t1,": ",loses,"-",wins)
    advance_to_second_round.append(t2)


Panathinaikos OPAP wins series against  Larisa :  2 - 0
AEK Athens wins series against  Iraklis B.C. :  2 - 0
Peristeri wins series against  Ifaistos Limnou :  2 - 1
Promitheas Patras wins series against  Lavrio B.C. :  2 - 1


## 2nd round

In [14]:
# first matchup
t1 = advance_to_second_round[0]
t2 = advance_to_second_round[3]
if standings.index(t1) > standings.index(t2):
    t1 = advance_to_second_round[3]
    t2 = advance_to_second_round[0]

advance_to_finals = []
small_finals = []
wins, loses = series_playoff(t1,t2,5,d)
if wins == 3:
    advance_to_finals.append(t1)
    small_finals.append(t2)
    print(t1,"wins series against ", t2,": ",wins,"-",loses)
else:
    print(t2,"wins series against ", t1,": ",loses,"-",wins)
    advance_to_finals.append(t2)
    small_finals.append(t1)
    
# second matchup
t1 = advance_to_second_round[1]
t2 = advance_to_second_round[2]
if standings.index(t1) > standings.index(t2):
    t1 = advance_to_second_round[3]
    t2 = advance_to_second_round[0]

wins, loses = series_playoff(t1,t2,5,d)
if wins == 3:
    advance_to_finals.append(t1)
    small_finals.append(t2)
    print(t1,"wins series against ", t2,": ",wins,"-",loses)
else:
    print(t2,"wins series against ", t1,": ",loses,"-",wins)
    advance_to_finals.append(t2)
    small_finals.append(t1)

Panathinaikos OPAP wins series against  Promitheas Patras :  3 - 1
AEK Athens wins series against  Peristeri :  3 - 0


In [15]:
# small final
t1 = small_finals[1]
t2 = small_finals[0]
if standings.index(t1) > standings.index(t2):
    t1 = small_finals[0]
    t2 = small_finals[1]

wins, loses = series_playoff(t1,t2,5,d)
if wins == 3:
    print(t1,"wins small final series against ", t2,": ",wins,"-",loses)
else:
    print(t2,"wins small final series against ", t1,": ",loses,"-",wins)

Peristeri wins small final series against  Promitheas Patras :  3 - 1


In [16]:
# finals
t1 = advance_to_finals[1]
t2 = advance_to_finals[0]
if standings.index(t1) > standings.index(t2):
    t1 = advance_to_finals[0]
    t2 = advance_to_finals[1]

wins, loses = series_playoff(t1,t2,5,d)
if wins == 3:
    print(t1,"wins finals series against ", t2,": ",wins,"-",loses)
else:
    print(t2,"wins finals series against ", t1,": ",loses,"-",wins)

Panathinaikos OPAP wins finals series against  AEK Athens :  3 - 0
